In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [2]:
#Reading the edit distance data
df = pd.read_csv('/kaggle/input/covid-10-edit.csv', delimiter=',', nrows = None)
df.head(10)

In [3]:
#Removing the genome name row from data
df1=df.iloc[: ,1: ]
df1.head(10)

In [4]:
#BONUS QUESTION
toy=pd.read_csv('/kaggle/input/toyseq-10.txt',sep=' ',header=None)
toy1=toy.drop([0],axis=1)

def edit_dist(s1,s2):
    n=len(s1)
    m=len(s2)
    mat_dist=np.zeros((n+1,m+1))
    for i in range(1,n+1):
        mat_dist[i,0]=i
    for j in range(m+1):
        mat_dist[0,j]=j
    for i in range(1,n+1):
        for j in range(1,m+1):
            if(s1[i-1]==s2[j-1]):
                #if character matches we do not add any "cost"
                mat_dist[i,j]=min(mat_dist[i-1,j-1],mat_dist[i-1,j]+1,mat_dist[i,j-1]+1)
            else:
                mat_dist[i,j]=min(mat_dist[i-1,j-1]+1,mat_dist[i-1,j]+1,mat_dist[i,j-1]+1)
    return mat_dist[n,m]

def edit_csv():
    dist_mat=[[]]
    for i in range(len(toy1[1])):
        for j in range(len(toy1[1])):
            dist_mat[i].append(edit_dist(toy1[1][i],toy1[1][j]))
        dist_mat.append([])
    dist_mat.pop()
    edit_dist_mat=pd.DataFrame(dist_mat,columns=None)
    edit_dist_mat.to_csv('toy_edit_distance.csv',index=False)
            

print("1st Sequence ",toy1[1][0])
print("2nd Sequence ",toy1[1][7])       
print("Edit Distance ",edit_dist(toy1[1][0],toy1[1][7]))
edit_csv()               

In [5]:
#Applying PCA to the edit distance matrix
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

pca=PCA(n_components=2)
principal_components=pca.fit_transform(df1)
pdf=pd.DataFrame(principal_components, columns=['pc1','pc2'])
plt.scatter(pdf.pc1,pdf.pc2)
plt.show()

In [6]:
#Normalizing the edit distance data
from sklearn.preprocessing import normalize
data_scaled=normalize(df1)
data_scaled=pd.DataFrame(data_scaled, columns=df1.columns)
data_scaled.head(10)

In [7]:
#Applying PCA to the normalized edit distance data
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

pca=PCA(n_components=2)
principal_components=pca.fit_transform(data_scaled)
pdf1=pd.DataFrame(principal_components, columns=['pc1','pc2'])
plt.scatter(pdf1.pc1,pdf1.pc2)
plt.show()

In [8]:
import scipy.cluster.hierarchy as hier
plt.figure(figsize=(5,5))
plt.title("Normalized MIN Dendrogram")
dendo=hier.dendrogram(hier.linkage(data_scaled,method='single'))

In [9]:
import scipy.cluster.hierarchy as hier
plt.figure(figsize=(5,5))
plt.title("MIN Dendogram")
dendo=hier.dendrogram(hier.linkage(df1,method='single'))

In [10]:
import scipy.cluster.hierarchy as hier
plt.figure(figsize=(5,5))
plt.title("MAX Dendogram")
dendo=hier.dendrogram(hier.linkage(df1,method='complete'))

In [11]:
import scipy.cluster.hierarchy as hier
plt.figure(figsize=(5,5))
plt.title("AVG Dendogram")
dendo=hier.dendrogram(hier.linkage(df1,method='average'))

<p> From the above clustering techniques, the average hierarchal clustering technique seems to cluster the genomes most accurately because when using the MAX or the MIN clustering techinques, the contribution of the cluster point farthest or closest to the new point is dominating, whereas when using the Average clustering technique, the similarity of all the cluster points is taken into account, thus the new point that will be added will be the point that is most similar to all other points already in the cluster, so using this technique will allow us to cluster all similar genomes more accurately than the others.</p>